# Car Price Prediction

In [ ]:
!wget https://www.dropbox.com/s/1rnguy1lh6su0p3/vehicles.csv.zip?dl=0
!unzip vehicles.csv.zip?dl=0

--2021-02-07 06:12:52--  https://www.dropbox.com/s/1rnguy1lh6su0p3/vehicles.csv.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/1rnguy1lh6su0p3/vehicles.csv.zip [following]
--2021-02-07 06:12:52--  https://www.dropbox.com/s/raw/1rnguy1lh6su0p3/vehicles.csv.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc296bd11b87542c9440ba9b1db5.dl.dropboxusercontent.com/cd/0/inline/BIe_aJnw-AiefYZ5FJzop-AyDQrosdbaMQ2mzTqab8RMSltsR3V-Le3elE6m4fJKUDqNpptR6kFgkSpwKsEpJ76Wv5I1EgU8ebODJ3Jm1yUrfOgHc0u_n_CK3dHPBQzoR00/file# [following]
--2021-02-07 06:12:52--  https://uc296bd11b87542c9440ba9b1db5.dl.dropboxusercontent.com/cd/0/inline/BIe_aJnw-AiefYZ5FJzop-AyDQrosdbaMQ2mzTqab8RMSltsR3V-Le3elE6m4fJKUDqNpptR6kFgkSpwKsEpJ76Wv5I1Eg

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.dummy import DummyRegressor
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-deep')
plt.rcParams.update({'font.size': 15})

In [ ]:
df = pd.read_csv('vehicles.csv')

In [ ]:
df.shape

(458213, 26)

In [ ]:
len(df[df.price == 0])

33753

## Data Cleaning

In [ ]:
# Removing rows with missing price and year
df.drop(df[df['price'] == 0].index, inplace = True) 
df.drop(df[df['year'].isna()].index, inplace = True) 
df.drop(df[df['year'] == 2021].index, inplace = True)
df.drop([496], inplace = True)
df.drop(df[df.odometer > 500000].index, inplace = True)

In [ ]:
# Changing type of year to int
df['year'] = df['year'].astype(int)
df['year'].dtype

dtype('int64')

In [ ]:
year_med =  dict(df.groupby('year')['odometer'].median())

In [ ]:
# Imputing missing values in odometer with median odometer of each year cars
df['odometer'] = df['odometer'].fillna(df['year'].apply(lambda x: year_med.get(x)))

In [ ]:
# Removing the leftover missing values in odometer because no information available for those years
df.drop(df[df['odometer'].isna()].index, inplace = True) 

In [ ]:
df.shape

(421812, 26)

In [ ]:
# Dropping cars with price less than 1000 with miles less than 60,000 and model year greater than 2010
df.drop(df[(df.price < 1000 ) & (df.odometer < 60000 ) & (df.year > 2010)].index, inplace = True)
df.drop(df[(df.price < 200)].index, inplace = True)
df.drop(df[(df.price > 50000)].index, inplace = True)

In [ ]:
df.shape

(405261, 26)

In [ ]:
df.head()

,Unnamed: 0,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,0,7240372487,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,35990,2010,chevrolet,corvette grand sport,good,8 cylinders,gas,32742.0,clean,other,1G1YU3DW1A5106980,rwd,NaN,other,NaN,https://images.craigslist.org/00N0N_ipkbHVZYf4...,Carvana is the safer way to buy a car During t...,al,32.590000,-85.480000,2020-12-02T08:11:30-0600
1,1,7240309422,https://auburn.craigslist.org/cto/d/auburn-201...,auburn,https://auburn.craigslist.org,7500,2014,hyundai,sonata,excellent,4 cylinders,gas,93600.0,clean,automatic,5NPEC4AB0EH813529,fwd,NaN,sedan,NaN,https://images.craigslist.org/00s0s_gBHYmJ5o7y...,I'll move to another city and try to sell my c...,al,32.547500,-85.468200,2020-12-02T02:11:50-0600
2,2,7240224296,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,4900,2006,bmw,x3 3.0i,good,6 cylinders,gas,87046.0,clean,automatic,NaN,NaN,NaN,SUV,blue,https://images.craigslist.org/00B0B_5zgEGWPOrt...,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,al,32.616807,-85.464149,2020-12-01T19:50:41-0600
3,3,7240103965,https://auburn.craigslist.org/cto/d/lanett-tru...,auburn,https://auburn.craigslist.org,2000,1974,chevrolet,c-10,good,4 cylinders,gas,190000.0,clean,automatic,NaN,rwd,full-size,pickup,blue,https://images.craigslist.org/00M0M_6o7KcDpArw...,1974 chev. truck (LONG BED) NEW starter front ...,al,32.861600,-85.216100,2020-12-01T15:54:45-0600
4,4,7239983776,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,19500,2005,ford,f350 lariat,excellent,8 cylinders,diesel,116000.0,lien,automatic,NaN,4wd,full-size,pickup,blue,https://images.craigslist.org/00p0p_b95l1EgUfl...,2005 Ford F350 Lariat (Bullet Proofed). This t...,al,32.547500,-85.468200,2020-12-01T12:53:56-0600


In [ ]:
# Dropping rows with missing cylinders, transmission and drive
df.drop(df[(df['cylinders'].isna()) & (df['transmission'].isna()) & (df['drive'].isna())].index, inplace = True)

In [ ]:
df['cylinders'] = df['cylinders'].apply(lambda x: x.split()[0] if not pd.isnull(x) else np.nan)

In [ ]:
# Changing cylinders of tesla cars to 0
df.loc[df.manufacturer == 'tesla', 'cylinders'] = 0
df.cylinders.fillna(0, inplace = True)
df.cylinders =  df.cylinders.replace('other', 0)

In [ ]:
df.columns

Index(['Unnamed: 0', 'id', 'url', 'region', 'region_url', 'price', 'year',
       'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer',
       'title_status', 'transmission', 'VIN', 'drive', 'size', 'type',
       'paint_color', 'image_url', 'description', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [ ]:
df['cylinders'] = df['cylinders'].astype(int)


In [ ]:
# Changing cylinder of all 2019 honda accords to 4
df.loc[((df.manufacturer == 'honda') & (df.year == 2019) & (df.model == 'accord')), 'cylinders'] = 4

In [ ]:
df.loc[((df.manufacturer == 'honda') & (df.year == 2020) & (df.model == 'accord')),'cylinders'] = 4

In [ ]:
# Dropping unwanted columns
df.drop(['id', 'url', 'region_url', 'VIN', 'lat', 'long','image_url', 'region'], axis = 1, inplace = True) 

In [ ]:
df.shape

(404292, 18)

In [ ]:
df.isna().sum()

Unnamed: 0           0
price                0
year                 0
manufacturer     13553
model             4092
condition       158026
cylinders            0
fuel              2106
odometer             0
title_status      1361
transmission       740
drive           116400
size            281950
type             99911
paint_color     119149
description         42
state                0
posting_date         0
dtype: int64

In [ ]:
# filling missing condition values with fair considering their condition would be fair
#df.condition.fillna('fair', inplace = True)

In [ ]:
df.condition.replace('like new', 'excellent', inplace = True)
df.condition.replace('new', 'excellent', inplace = True)

In [ ]:
df.condition.value_counts()

excellent    127147
good         110967
fair           7546
salvage         606
Name: condition, dtype: int64

In [ ]:
df.fuel.fillna('gas', inplace = True)

In [ ]:
df.fuel.value_counts()

gas         361438
diesel       23312
other        12310
hybrid        5176
electric      2056
Name: fuel, dtype: int64

In [ ]:
df.transmission.value_counts()

automatic    349195
other         29694
manual        24663
Name: transmission, dtype: int64

In [ ]:
df["description"]=df["description"].apply(lambda x:str(x).lower())
df.loc[(df['description'].str.contains('automatic transmission')) & (df['transmission'] == 'other'), "transmission"] = "automatic"
df.loc[(df['description'].str.contains('manual transmission')) & (df['transmission'] == 'other'), "transmission"] = "manual"
df.loc[(df['description'].str.contains('automatic transmission')) & (df['transmission'].isna()), "transmission"] = "automatic"
df.loc[(df['description'].str.contains('manual transmission')) & (df['transmission'].isna()), "transmission"] = "manual"

In [ ]:
# Assuming most cars have automatic transmission
df.transmission.fillna('automatic', inplace = True)

In [ ]:
df.isna().mean()

Unnamed: 0      0.000000
price           0.000000
year            0.000000
manufacturer    0.033523
model           0.010121
condition       0.390871
cylinders       0.000000
fuel            0.000000
odometer        0.000000
title_status    0.003366
transmission    0.000000
drive           0.287911
size            0.697392
type            0.247126
paint_color     0.294710
description     0.000000
state           0.000000
posting_date    0.000000
dtype: float64

In [ ]:
# Creating age from year
df['age'] = 2021 - df['year']
#df.drop(columns = 'year', inplace = True)

In [ ]:
df.drop(df[df.odometer < 10].index, inplace = True) 
df.drop(df[df.odometer > 300000].index, inplace = True)

In [ ]:
df.shape

(400724, 19)

In [ ]:
dff = df

In [ ]:
df.columns

Index(['Unnamed: 0', 'price', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission',
       'drive', 'size', 'type', 'paint_color', 'description', 'state',
       'posting_date', 'age'],
      dtype='object')

In [ ]:
target = df['price']

In [ ]:
df.shape

(400724, 19)

In [ ]:
# Going through description to check how many cars have new tires and creating a column for new tires
df['new_tires'] = np.where((df['description'].str.contains('new tire')), 1,0) 

In [ ]:
df['new_tires'].sum()

26111

In [ ]:
df['heated_seats'] = np.where((df['description'].str.contains('HEATED SEAT')) |
                           (df['description'].str.contains('heated seat')) |
                           (df['description'].str.contains('Heated Seat')) |
                           (df['description'].str.contains('heated SEAT')) |
                           (df['description'].str.contains('HEATED seat')), 1,0)

In [ ]:
df['heated_seats'].sum()

37347

In [ ]:
df['auto_headlight'] = np.where((df['description'].str.contains('automatic headlight')) |
                                (df['description'].str.contains('auto headlight')) |
                                (df['description'].str.contains('AUTOMATIC HEADLIGHT')) |
                                (df['description'].str.contains('Automatic Headlight')) |
                                (df['description'].str.contains('AUTO HEADLIGHT')) |
                                (df['description'].str.contains('Auto Headlight')), 1,0)

In [ ]:
df['auto_headlight'].sum()

20405

In [ ]:
df['leather'] = np.where((df['description'].str.contains('leather')) |
                          (df['description'].str.contains('Leather'))|
                          (df['description'].str.contains('LEATHER')) , 1,0)

In [ ]:
df['leather'].sum()

102424

In [ ]:
# Removing Description and model column
df.drop(['description', 'model','state'], axis = 1, inplace = True)


In [ ]:
df.columns

Index(['Unnamed: 0', 'price', 'year', 'manufacturer', 'condition', 'cylinders',
       'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'size',
       'type', 'paint_color', 'posting_date', 'age', 'new_tires',
       'heated_seats', 'auto_headlight', 'leather'],
      dtype='object')

## Baseline Model

In [ ]:
dff.columns
base_features = [ 'odometer','age', 'new_tires', 'heated_seats', 'auto_headlight', 'leather', 'cylinders'] 

In [ ]:
scaler = StandardScaler()
#instantiate a linear regression object
scaler.fit(dff[base_features])
lm = linear_model.LinearRegression()

#fit the linear regression to the data
lm = lm.fit(dff[base_features], target)


print(lm.intercept_)
print(lm.coef_)
print ("R^2 Score:", lm.score(dff[base_features], target))
pred = lm.predict(dff[base_features])
rmse = np.sqrt(metrics.mean_squared_error(target, pred))
print('Root Mean Squared Error:' , rmse)

25282.078769157433
[-9.30525778e-02 -3.04349494e+02 -1.82710248e+03  7.97978127e+02
 -2.03259557e+02  1.31904078e+03  5.11230377e+02]
R^2 Score: 0.38220161850182444
Root Mean Squared Error: 8842.287904018494


In [ ]:
df.columns

Index(['Unnamed: 0', 'price', 'year', 'manufacturer', 'condition', 'cylinders',
       'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'size',
       'type', 'paint_color', 'posting_date', 'age', 'new_tires',
       'heated_seats', 'auto_headlight', 'leather'],
      dtype='object')

In [ ]:
target = df['price']
df =  pd.get_dummies(df, columns=['condition'], drop_first = True )
df =  pd.get_dummies(df, columns=['fuel'], drop_first = True )
df =  pd.get_dummies(df, columns=['title_status'], drop_first = True )
df =  pd.get_dummies(df, columns=['transmission'], drop_first = True )
df =  pd.get_dummies(df, columns=['drive'], drop_first = True )
df =  pd.get_dummies(df, columns=['size'], drop_first = True )
df =  pd.get_dummies(df, columns=['type'], drop_first = True )
df =  pd.get_dummies(df, columns=['paint_color'], drop_first = True )
#df =  pd.get_dummies(df, columns=['state'], drop_first = True )
#df =  pd.get_dummies(df, columns=['cylinders'], drop_first = True )
df =  pd.get_dummies(df, columns=['manufacturer'], drop_first = True )

In [ ]:
df.head()

,Unnamed: 0,price,year,cylinders,odometer,posting_date,age,new_tires,heated_seats,auto_headlight,leather,condition_fair,condition_good,condition_salvage,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,transmission_manual,transmission_other,drive_fwd,drive_rwd,size_full-size,size_mid-size,size_sub-compact,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_datsun,manufacturer_dodge,manufacturer_ferrari,manufacturer_fiat,manufacturer_ford,manufacturer_gmc,manufacturer_harley-davidson,manufacturer_hennessey,manufacturer_honda,manufacturer_hyundai,manufacturer_infiniti,manufacturer_jaguar,manufacturer_jeep,manufacturer_kia,manufacturer_land rover,manufacturer_lexus,manufacturer_lincoln,manufacturer_mazda,manufacturer_mercedes-benz,manufacturer_mercury,manufacturer_mini,manufacturer_mitsubishi,manufacturer_nissan,manufacturer_pontiac,manufacturer_porsche,manufacturer_ram,manufacturer_rover,manufacturer_saturn,manufacturer_subaru,manufacturer_tesla,manufacturer_toyota,manufacturer_volkswagen,manufacturer_volvo
0,0,35990,2010,8,32742.0,2020-12-02T08:11:30-0600,11,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,7500,2014,4,93600.0,2020-12-02T02:11:50-0600,7,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,4900,2006,6,87046.0,2020-12-01T19:50:41-0600,15,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2000,1974,4,190000.0,2020-12-01T15:54:45-0600,47,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,19500,2005,8,116000.0,2020-12-01T12:53:56-0600,16,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.shape

(400724, 94)

In [ ]:
df.columns

Index(['Unnamed: 0', 'price', 'year', 'cylinders', 'odometer', 'posting_date',
       'age', 'new_tires', 'heated_seats', 'auto_headlight', 'leather',
       'condition_fair', 'condition_good', 'condition_salvage',
       'fuel_electric', 'fuel_gas', 'fuel_hybrid', 'fuel_other',
       'title_status_lien', 'title_status_missing', 'title_status_parts only',
       'title_status_rebuilt', 'title_status_salvage', 'transmission_manual',
       'transmission_other', 'drive_fwd', 'drive_rwd', 'size_full-size',
       'size_mid-size', 'size_sub-compact', 'type_bus', 'type_convertible',
       'type_coupe', 'type_hatchback', 'type_mini-van', 'type_offroad',
       'type_other', 'type_pickup', 'type_sedan', 'type_truck', 'type_van',
       'type_wagon', 'paint_color_blue', 'paint_color_brown',
       'paint_color_custom', 'paint_color_green', 'paint_color_grey',
       'paint_color_orange', 'paint_color_purple', 'paint_color_red',
       'paint_color_silver', 'paint_color_white', 'paint_color_y

In [ ]:
def train_regression_model(df,kfold):
  features = ['age', 'odometer', 'new_tires', 'heated_seats',
       'auto_headlight', 'leather', 'cylinders',
       'condition_fair', 'condition_good', 'condition_salvage',
       'fuel_electric', 'fuel_gas', 'fuel_hybrid', 'fuel_other',
       'title_status_lien', 'title_status_missing', 'title_status_parts only',
       'title_status_rebuilt', 'title_status_salvage', 'transmission_manual',
       'transmission_other', 'drive_fwd', 'drive_rwd', 'size_full-size',
       'size_mid-size', 'size_sub-compact', 'type_bus', 'type_convertible',
       'type_coupe', 'type_hatchback', 'type_mini-van', 'type_offroad',
       'type_other', 'type_pickup', 'type_sedan', 'type_truck', 'type_van',
       'type_wagon', 'paint_color_blue', 'paint_color_brown',
       'paint_color_custom', 'paint_color_green', 'paint_color_grey',
       'paint_color_orange', 'paint_color_purple', 'paint_color_red',
       'paint_color_silver', 'paint_color_white', 'paint_color_yellow']

  scaler = StandardScaler()
  scaler2 = MinMaxScaler()
  #call train_test_split on the data and capture the results
  train_data=df[df.kfold != fold].reset_index(drop=True)
  valid_data=df[df.kfold == fold].reset_index(drop=True)
  X_train=train_data[features]
  y_train=train_data["price"]
  X_test=valid_data[features]
  y_test=valid_data["price"]
  #print(len(y_test.values),len(valid_data),len(X_test))
  scaler2.fit(X_train)
  lm = linear_model.LinearRegression()
  lm = lm.fit(X_train, y_train)
  y_train_pred = lm.predict(X_train)
  train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
  print('Root Mean Squared Error:' , train_rmse)
  y_pred = lm.predict(X_test)
  test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
  print('Root Mean Squared Error:' + str(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))
  print('Fold: ',int(kfold),'Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))
  return train_rmse,test_rmse

In [ ]:
from sklearn import model_selection
df["kfold"]=-1
df=df.sample(frac=1).reset_index(drop=True)
kf=model_selection.KFold(n_splits=5)
for fold,(tar,val)in enumerate(kf.split(df)):
  df.loc[val,"kfold"]=fold

In [ ]:
avg_train_rmse=0
avg_test_rmse=0
for i in range(5):
  print("="*50)
  train_rmse,test_rmse=train_regression_model(df,i)
  avg_train_rmse+=train_rmse
  avg_test_rmse+=test_rmse
  print("="*50)
print("Average Train RMSE : ",avg_train_rmse/5)
print("Average Test RMSE : ",avg_test_rmse/5)

Root Mean Squared Error: 7221.746798715325
Root Mean Squared Error:7260.819950667665
Fold:  0 Training:  7221 vs. Testing:  7260
Root Mean Squared Error: 7221.746798715325
Root Mean Squared Error:7260.819950667665
Fold:  1 Training:  7221 vs. Testing:  7260
Root Mean Squared Error: 7221.746798715325
Root Mean Squared Error:7260.819950667665
Fold:  2 Training:  7221 vs. Testing:  7260
Root Mean Squared Error: 7221.746798715325
Root Mean Squared Error:7260.819950667665
Fold:  3 Training:  7221 vs. Testing:  7260
Root Mean Squared Error: 7221.746798715325
Root Mean Squared Error:7260.819950667665
Fold:  4 Training:  7221 vs. Testing:  7260
Average Train RMSE :  7221.746798715324
Average Test RMSE :  7260.819950667664


In [ ]:
# testing the model on training data and getting the rmse
df

,Unnamed: 0,price,year,cylinders,odometer,posting_date,age,new_tires,heated_seats,auto_headlight,leather,condition_fair,condition_good,condition_salvage,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,transmission_manual,transmission_other,drive_fwd,drive_rwd,size_full-size,size_mid-size,size_sub-compact,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_datsun,manufacturer_dodge,manufacturer_ferrari,manufacturer_fiat,manufacturer_ford,manufacturer_gmc,manufacturer_harley-davidson,manufacturer_hennessey,manufacturer_honda,manufacturer_hyundai,manufacturer_infiniti,manufacturer_jaguar,manufacturer_jeep,manufacturer_kia,manufacturer_land rover,manufacturer_lexus,manufacturer_lincoln,manufacturer_mazda,manufacturer_mercedes-benz,manufacturer_mercury,manufacturer_mini,manufacturer_mitsubishi,manufacturer_nissan,manufacturer_pontiac,manufacturer_porsche,manufacturer_ram,manufacturer_rover,manufacturer_saturn,manufacturer_subaru,manufacturer_tesla,manufacturer_toyota,manufacturer_volkswagen,manufacturer_volvo,kfold
0,31619,649,2017,0,101977.0,2020-11-28T19:34:37-0800,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,154360,7995,2014,4,105632.0,2020-11-19T11:49:32-0600,7,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,394259,38500,2014,0,130087.0,2020-11-20T21:05:33-0700,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,203626,3500,2008,6,145000.0,2020-11-27T18:07:33-0500,13,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,159971,17995,2018,6,56105.0,2020-11-07T08:07:41-0600,3,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400719,152430,2999,2007,6,187000.0,2020-12-02T17:00:45-0600,14,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
400720,452731,3998,2000,8,276223.0,2020-12-03T08:45:25-0700,21,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
400721,88057,1995,2012,6,102268.0,2020-11-13T11:31:26-0500,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
400722,267056,15950,2006,0,145000.0,2020-11-30T11:46:34-0700,15,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))

Training:  7221 vs. Testing:  7260


### Using k best

In [ ]:
selector = SelectKBest(f_regression, k = 20)
selector.fit(X_train, y_train)

selected_columns = X_train.columns[selector.get_support()]
removed_columns = X_train.columns[~selector.get_support()]

In [ ]:
list(removed_columns)

['new_tires heated_seats', 'new_tires auto_headlight']

In [ ]:
list(selected_columns)

['odometer',
 'age',
 'heated_seats',
 'auto_headlight',
 'leather',
 'odometer new_tires',
 'odometer heated_seats',
 'odometer auto_headlight',
 'odometer leather',
 'age new_tires',
 'age leather',
 'age cylinders',
 'new_tires leather',
 'new_tires cylinders',
 'heated_seats auto_headlight',
 'heated_seats leather',
 'heated_seats cylinders',
 'auto_headlight leather',
 'auto_headlight cylinders',
 'leather cylinders']

In [ ]:
#instantiate a linear regression object
lm_kbest = LinearRegression()

#fit the linear regression to the data
lm_kbest = lm_kbest.fit(X_train[selected_columns], y_train)

y_train_kbest = lm_kbest.predict(X_train[selected_columns])


trainK_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_kbest))


print('Training Root Mean Squared Error:' , trainK_rmse)

y_kbest = lm_kbest.predict(X_test[selected_columns])

testK_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_kbest))

print('Testing Root Mean Squared Error:' , testK_rmse)


print('Original: ', test_rmse, "vs. KBest: ", testK_rmse)
lm_kbest.score(X_train[selected_columns], y_train)

Training Root Mean Squared Error: 8877.702232075446
Testing Root Mean Squared Error: 8868.081457248829
Original:  7260.819950667665 vs. KBest:  8868.081457248829


0.3774595606015513